# Data pre process

In [61]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, learning_curve, RandomizedSearchCV,GridSearchCV
from sklearn.metrics import precision_score, recall_score, roc_auc_score, fbeta_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import keras.backend as K
from keras.optimizers import SGD, Adam 
from keras.models import load_model
from keras import regularizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


import warnings
warnings.filterwarnings("ignore")

In [2]:
dt_path = "../data/kdd2009/"

In [3]:
#first 190 features are numerical and the last 40 are categorical
X = pd.read_table(dt_path+"orange_small_train.data")
X.shape

(50000, 230)

In [4]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [5]:
#X = reduce_mem_usage(X)

## features transformation

In [6]:
#X = reduce_mem_usage(X)

In [7]:
num_features = list(X.columns[:190])
cat_features = list(X.columns[190:230])

In [8]:
empty_features = []
c = 0.3
for feat in X.columns:
    nulls = X[feat].isnull().value_counts()
    try:
        not_nulls = nulls[False]
        if not_nulls < c*40000:
            empty_features.append(feat)
    except:    
        empty_features.append(feat)
print ("number of empty features is", len(empty_features))

number of empty features is 154


In [9]:
#remove sparse features
for feat in empty_features:
    #data.drop(feat, axis = 1, inplace = True)    
    if feat in num_features:
        num_features.remove(feat)
    else:
        cat_features.remove(feat)
    X.drop(feat,axis=1, inplace=True)

## missing value imputation

In [10]:
#Numeric features: replace missing values with (max value + 1).
maxs = X.max(axis = 0)
for i,feat in enumerate(num_features):
    fill_value = maxs[i] + 1.
    X.fillna({feat: fill_value}, inplace=True)

In [11]:
#Categorial features: replace values with their frequencies.
data_cat_all = X[cat_features]

for feat in data_cat_all.columns:
    data_cat_all[feat] = data_cat_all[feat].map(data_cat_all.groupby(feat).size())
    
X[cat_features] = data_cat_all.loc[:39999,:]

In [12]:
#Categorial features: replace missing values with zeros.
X.fillna(0., inplace=True)

## Load target valuable

In [13]:
y = pd.read_table(dt_path+"orange_small_train_churn.labels",header=None)
y.shape

(50000, 1)

In [14]:
y = LabelEncoder().fit_transform(y)

In [31]:
pd.Series(y).value_counts() 

0    46328
1     3672
dtype: int64

## get data ready for training

In [62]:
Scaler = MinMaxScaler()
X = pd.DataFrame(Scaler.fit_transform(X),columns=X.columns,index=X.index)

In [63]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=2018, stratify=y)

# Modeling

## Single Sequential Model

In [64]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import keras.backend as K
from keras.optimizers import SGD, Adam 
from keras.models import load_model
from keras import regularizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [112]:
NB_EPOCH = 200
BATCH_SIZE = 4096
VERBOSE = 1
OPTIMIZER = 'adam'

In [113]:
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1],kernel_initializer='random_uniform'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(16, input_dim=X_train.shape[1],kernel_initializer='random_uniform'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(8, kernel_initializer='random_uniform'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1,kernel_initializer='random_uniform'))
model.add(Activation('sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 32)                2464      
_________________________________________________________________
activation_21 (Activation)   (None, 32)                0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 16)                528       
_________________________________________________________________
activation_22 (Activation)   (None, 16)                0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 8)                 136       
__________

In [114]:
model.compile(optimizer=OPTIMIZER,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [115]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')

In [116]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0001, patience=20, verbose=0, mode='auto'),
    # EarlyStopping(monitor='val_loss', patience=2, verbose=0),
    #ModelCheckpoint(kfold_weights_path, monitor='val_loss', save_best_only=True, verbose=0),
]

In [123]:
model.fit(np.array(X_train), np.array(y_train), epochs=NB_EPOCH, batch_size=BATCH_SIZE,callbacks=callbacks)

Epoch 1/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2567 - acc: 0.9266
Epoch 2/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2577 - acc: 0.9266
Epoch 3/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2582 - acc: 0.9266
Epoch 4/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2559 - acc: 0.9265
Epoch 5/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2554 - acc: 0.9265
Epoch 6/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2570 - acc: 0.9265
Epoch 7/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2562 - acc: 0.9266
Epoch 8/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2560 - acc: 0.9265
Epoch 9/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2551 - acc: 0.9266
Epoch 10/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2553 - acc: 0.9265

40000/40000 [==============================] - 0s 3us/step - loss: 0.2523 - acc: 0.9265
Epoch 83/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2534 - acc: 0.9265
Epoch 84/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2528 - acc: 0.9266
Epoch 85/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2529 - acc: 0.9265
Epoch 86/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2516 - acc: 0.9265
Epoch 87/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2520 - acc: 0.9265
Epoch 88/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2515 - acc: 0.9265
Epoch 89/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2540 - acc: 0.9265
Epoch 90/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2532 - acc: 0.9266
Epoch 91/200
40000/40000 [==============================] - 0s 3us/step - loss: 0.2513 - acc: 0.9265
Epo

In [119]:
# training performance
y_pred_proba = model.predict_proba(np.array(X_test), batch_size=512)

In [120]:
roc_auc_score(y_test, y_pred_proba)

0.7070505042900976

In [121]:
flattened = [val for sublist in y_pred_proba for val in sublist]
pd.Series(flattened).describe()

## Cross Validation

In [128]:
NB_EPOCH = 200
BATCH_SIZE = 4096
OPTIMIZER = 'adam'
K_FOLD = 10
LAMBD = 0.002 #0.000005
SEED = 12#np.random.randn()

kfold = StratifiedKFold(n_splits=K_FOLD, shuffle=False, random_state=SEED)
models = []
cvscores_training = []
cvscores_validaton = []
ind = 0.4

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=15, epsilon=0.0001, min_lr=0.0000001)
earlyStopping = EarlyStopping(monitor='loss', min_delta=0.00001, patience=40, verbose=0, mode='auto')

callbacks = [reduce_lr,earlyStopping]

for train, test in kfold.split(X, y):
    
    model = Sequential()
    model.add(Dense(100, input_dim=X.shape[1],kernel_initializer='random_uniform'))
    model.add(Activation('relu'))
    model.add(Dropout(0.5, seed = SEED))
    model.add(Dense(50, kernel_initializer='random_uniform',kernel_regularizer=regularizers.l2(LAMBD)))
    model.add(Activation('relu'))
    model.add(Dropout(0.3, seed = SEED))
    model.add(Dense(16, kernel_initializer='random_uniform',kernel_regularizer=regularizers.l2(LAMBD)))
    model.add(Activation('relu'))
    model.add(Dropout(0.2, seed = SEED))
    model.add(Dense(8, kernel_initializer='random_uniform'))
    model.add(Activation('relu'))
    #model.add(Dropout(0.1, seed = SEED))
    model.add(Dense(1,kernel_initializer='random_uniform'))
    model.add(Activation('sigmoid'))
    #model.summary()  
    model.compile(optimizer=OPTIMIZER, loss='binary_crossentropy', metrics=['accuracy'])
    
    print('\n model {} starts to fit ... \n'.format(ind))
    model.fit(np.array(X.iloc[train]), np.array(y[train]), epochs=NB_EPOCH, batch_size=BATCH_SIZE, validation_split=0, callbacks=callbacks)
    
    # evaluate the model    
    p_training = model.predict_proba(np.array(X.iloc[train]), batch_size=4096, verbose=0)
    score_training = roc_auc_score(y[train],p_training)
    p_validaton = model.predict_proba(np.array(X.iloc[test]), batch_size=4096, verbose=0)
    score_validaton = roc_auc_score(y[test],p_validaton)
    print("\n AUC: training is {}; validation is {}".format(score_training,score_validaton))
        
    cvscores_training.append(score_training)
    cvscores_validaton.append(score_validaton)
    
#    model_path = '../../models/DL_V2.0/' + 'CV_model_' + str(ind)
#    model.save(model_path)
#    models.append(model_path)
    
    ind += 1
    
print("training AUC: maximum: {}; minimun: {}; mean: {}; sd: {}".format \
      (np.max(cvscores_training), np.min(cvscores_training), np.mean(cvscores_training), np.std(cvscores_training)))
print("validaton AUC: maximum: {}; minimun: {}; mean: {}; sd: {}".format \
      (np.max(cvscores_validaton), np.min(cvscores_validaton), np.mean(cvscores_validaton), np.std(cvscores_validaton)))


 model 0.4 starts to fit ... 

Epoch 1/200
44999/44999 [==============================] - 1s 15us/step - loss: 0.6978 - acc: 0.9260
Epoch 2/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.6873 - acc: 0.9266
Epoch 3/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.6733 - acc: 0.9266
Epoch 4/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.6145 - acc: 0.9266
Epoch 5/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.4142 - acc: 0.9266
Epoch 6/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.3363 - acc: 0.9266
Epoch 7/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.3193 - acc: 0.9266
Epoch 8/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.3101 - acc: 0.9266
Epoch 9/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.3026 - acc: 0.9266
Epoch 10/200
44999/44999 [==============================] - 0s 4us/

44999/44999 [==============================] - 0s 4us/step - loss: 0.2520 - acc: 0.9266
Epoch 82/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2514 - acc: 0.9266
Epoch 83/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2528 - acc: 0.9266
Epoch 84/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2520 - acc: 0.9266
Epoch 85/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2520 - acc: 0.9266
Epoch 86/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2520 - acc: 0.9266
Epoch 87/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2517 - acc: 0.9266
Epoch 88/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2519 - acc: 0.9266
Epoch 89/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2516 - acc: 0.9266
Epoch 90/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2519 - acc: 0.9266
Epo

44999/44999 [==============================] - 0s 4us/step - loss: 0.2497 - acc: 0.9266
Epoch 162/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2495 - acc: 0.9266
Epoch 163/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2494 - acc: 0.9266
Epoch 164/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2502 - acc: 0.9266
Epoch 165/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2490 - acc: 0.9266
Epoch 166/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2502 - acc: 0.9266
Epoch 167/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2496 - acc: 0.9266
Epoch 168/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2495 - acc: 0.9266
Epoch 169/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2494 - acc: 0.9266
Epoch 170/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2494 - acc: 0

44999/44999 [==============================] - 0s 4us/step - loss: 0.2616 - acc: 0.9266
Epoch 41/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2607 - acc: 0.9266
Epoch 42/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2587 - acc: 0.9266
Epoch 43/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2604 - acc: 0.9266
Epoch 44/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2589 - acc: 0.9266
Epoch 45/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2606 - acc: 0.9266
Epoch 46/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2590 - acc: 0.9266
Epoch 47/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2590 - acc: 0.9266
Epoch 48/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2585 - acc: 0.9266
Epoch 49/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2583 - acc: 0.9266
Epo

44999/44999 [==============================] - 0s 4us/step - loss: 0.2515 - acc: 0.9266
Epoch 122/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2511 - acc: 0.9266
Epoch 123/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2519 - acc: 0.9266
Epoch 124/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2517 - acc: 0.9266
Epoch 125/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2514 - acc: 0.9266
Epoch 126/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2509 - acc: 0.9266
Epoch 127/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2518 - acc: 0.9266
Epoch 128/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2512 - acc: 0.9266
Epoch 129/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2509 - acc: 0.9266
Epoch 130/200
44999/44999 [==============================] - 0s 4us/step - loss: 0.2512 - acc: 0

...

45001/45001 [==============================] - 1s 22us/step - loss: 0.6978 - acc: 0.9058
Epoch 2/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.6879 - acc: 0.9266
Epoch 3/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.6756 - acc: 0.9266
Epoch 4/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.6281 - acc: 0.9266
Epoch 5/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.4487 - acc: 0.9266
Epoch 6/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.3187 - acc: 0.9266
Epoch 7/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.3169 - acc: 0.9266
Epoch 8/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.3056 - acc: 0.9266
Epoch 9/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.3008 - acc: 0.9266
Epoch 10/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2971 - acc: 0.9266
Epoch 11/2

45001/45001 [==============================] - 0s 4us/step - loss: 0.2522 - acc: 0.9266
Epoch 83/200
45001/45001 [==============================] - 0s 5us/step - loss: 0.2519 - acc: 0.9266
Epoch 84/200
45001/45001 [==============================] - 0s 5us/step - loss: 0.2523 - acc: 0.9266
Epoch 85/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2523 - acc: 0.9266
Epoch 86/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2529 - acc: 0.9266
Epoch 87/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2526 - acc: 0.9266
Epoch 88/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2521 - acc: 0.9266
Epoch 89/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2520 - acc: 0.9266
Epoch 90/200
45001/45001 [==============================] - 0s 5us/step - loss: 0.2519 - acc: 0.9266
Epoch 91/200
45001/45001 [==============================] - 0s 5us/step - loss: 0.2516 - acc: 0.9266
Epo

45001/45001 [==============================] - 0s 4us/step - loss: 0.2488 - acc: 0.9266
Epoch 163/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2486 - acc: 0.9266
Epoch 164/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2488 - acc: 0.9266
Epoch 165/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2486 - acc: 0.9266
Epoch 166/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2487 - acc: 0.9266
Epoch 167/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2485 - acc: 0.9266
Epoch 168/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2484 - acc: 0.9266
Epoch 169/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2485 - acc: 0.9266
Epoch 170/200
45001/45001 [==============================] - 0s 5us/step - loss: 0.2485 - acc: 0.9266
Epoch 171/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2485 - acc: 0

45001/45001 [==============================] - 0s 4us/step - loss: 0.2593 - acc: 0.9266
Epoch 42/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2576 - acc: 0.9266
Epoch 43/200
45001/45001 [==============================] - 0s 5us/step - loss: 0.2566 - acc: 0.9266
Epoch 44/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2572 - acc: 0.9266
Epoch 45/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2572 - acc: 0.9266
Epoch 46/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2570 - acc: 0.9266
Epoch 47/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2566 - acc: 0.9266
Epoch 48/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2558 - acc: 0.9266
Epoch 49/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2558 - acc: 0.9266
Epoch 50/200
45001/45001 [==============================] - 0s 5us/step - loss: 0.2562 - acc: 0.9266
Epo

45001/45001 [==============================] - 0s 5us/step - loss: 0.2502 - acc: 0.9266
Epoch 123/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2505 - acc: 0.9266
Epoch 124/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2505 - acc: 0.9266
Epoch 125/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2497 - acc: 0.9266
Epoch 126/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2506 - acc: 0.9266
Epoch 127/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2498 - acc: 0.9266
Epoch 128/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2499 - acc: 0.9266
Epoch 129/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2495 - acc: 0.9266
Epoch 130/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2495 - acc: 0.9266
Epoch 131/200
45001/45001 [==============================] - 0s 4us/step - loss: 0.2505 - acc: 0